# Plipify

## Aim of this notebook

- Create fingerprints for individual structures of the same protein
- Create a frequency fingerprint across multiple structures of the same protein
- Visualize the frequency fingerpint

### Next to-dos:
- check variable names
- wrap code in functions and move to .py file
- path parameters

## Data Import and Preparation
### Import Packages

In [26]:
import csv
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from plip.modules.preparation import PDBComplex
from plip.modules.report import BindingSiteReport
from IPython.display import display, Markdown

### Pre-defined residues


In [27]:
with open('./data/MPro_residues.csv') as file:
    residue_reader = csv.reader(file, delimiter=',')
    residues = residue_reader.next()
residues = map(int,residues)

# consider other formats?

In [28]:
interaction_types = ["hydrophobic", "hbond", "waterbridge", "saltbridge", "pistacking", "pication", "halogen", "metal"] 
fp_length = len(residues)*len(interaction_types) # fp length based on number of pre-defined residues
fp = [0]*fp_length
fp_length

544

Create Index dictionaries for fingerprints

In [29]:
# split list l into lists of size n
def divide_list(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
        
# residue index dictionary
residue_i = range(0,fp_length)
residue_i = list(divide_list(residue_i,len(interaction_types)))
residue_dict = dict(zip(residues, residue_i))

# interaction_type index dictionary
interaction_i = range(0,len(interaction_types))
interaction_dict = dict(zip(interaction_types, interaction_i))

### Get PLIP data
From C-plip-fingerprints prototype

In [30]:
data = "./data/diamond_xchem_screen_mpro_all_pdbs"
with open(os.path.join(data, "diamond_xchem_screen_mpro_non_covalent_pdbs.dat")) as f:
    non_covalent_filenames = [l.strip() for l in f if l.strip()]

In [31]:
def analyze_interactions(pdbfile):
    protlig = PDBComplex()
    protlig.load_pdb(pdbfile)  # load the pdb
    for ligand in protlig.ligands:
        protlig.characterize_complex(ligand)  # find ligands and analyze interactions
    sites = {}
    for key, site in sorted(protlig.interaction_sets.items()):
        binding_site = BindingSiteReport(site)  # collect data about interactions
        # tuples of *_features and *_info will be converted to pandas df
        keys = "hydrophobic", "hbond", "waterbridge", "saltbridge", "pistacking", "pication", "halogen", "metal"
        interactions = {k: [getattr(binding_site, k+"_features")] + getattr(binding_site, k+"_info") for k in keys}
        sites[key] = interactions
    return sites

# for displaying data
def site_to_dataframes(site):
    keys = ["hydrophobic", "hbond", "waterbridge", "saltbridge", "pistacking", "pication", "halogen", "metal"] 
    dfs = {}
    for key in keys:
        records = site[key][1:]
        if not records:
            dfs[key] = None
        else:
            dfs[key] = pd.DataFrame.from_records(records, columns=site[key][0])
    return dfs

In [32]:
interactions = {}
for filename in tqdm(non_covalent_filenames):
    full_filename = os.path.join(data, filename)
    if not os.path.isfile(full_filename):
        print("File", full_filename, "does not exist?")
        continue
    interactions[filename] = analyze_interactions(full_filename)

('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0397.pdb', 'does not exist?')
('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0426.pdb', 'does not exist?')
('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0395.pdb', 'does not exist?')



In [33]:
for structure, sites in interactions.items():
    display(Markdown("# Structure {}".format(structure)))
    for site_name, site_interactions in sites.items():
        if not site_name.startswith('LIG'):
            continue  # fragments are labeled as LIG; other "sites" detected by PLIP are XRC artefacts
        display(Markdown("## Site {}".format(site_name)))
        for interaction_type, dataframe in site_to_dataframes(site_interactions).items():
            if dataframe is not None:
                display(Markdown("### {}".format(interaction_type)))
                display(dataframe)

# Structure Mpro-x0946.pdb

## Site LIG:A:1101

### waterbridge

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST_A-W,DIST_D-W,DON_ANGLE,WATER_ANGLE,PROTISDON,DONOR_IDX,DONORTYPE,ACCEPTOR_IDX,ACCEPTORTYPE,WATER_IDX,LIGCOO,PROTCOO,WATERCOO
0,166,GLU,A,1101,LIG,A,4.03,3.11,173.92,93.79,True,1288,Nam,2383,N3,2434,"(9.554, 5.104, 22.977)","(9.904, 2.716, 18.584)","(9.307, 1.422, 21.348)"


### hbond

...

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,SIDECHAIN,DIST_H-A,DIST_D-A,DON_ANGLE,PROTISDON,DONORIDX,DONORTYPE,ACCEPTORIDX,ACCEPTORTYPE,LIGCOO,PROTCOO
0,41,HIS,A,1101,LIG,A,True,2.44,3.08,122.60,True,309,Nar,2444,N1,"(14.061, -1.841, 20.54)","(12.794, -4.607, 21.042)"


### pistacking

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,CENTDIST,ANGLE,OFFSET,TYPE,LIG_IDX_LIST,LIGCOO,PROTCOO
0,41,HIS,A,1101,LIG,A,5.14,87.37,1.26,T,"2436,2437,2438,2439,2441,2442","(11.690833333333332, -0.2608333333333333, 23.2...","(11.795599999999999, -5.0338, 21.373)"


### hydrophobic

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST,LIGCARBONIDX,PROTCARBONIDX,LIGCOO,PROTCOO
0,189,GLN,A,1101,LIG,A,3.85,2437,1482,"(12.014, 0.979, 23.834)","(12.876, 2.521, 27.25)"


## Create Fingerprint
### Individual fingerprints
- create one count fp for each structure
- length of fp = length of pre-defined residues * number of interaction types

<img src = "./data/ifps.png">

In [34]:
# function to create one indivual fingerprint for all interactions of one structure
def create_fingerprint(residue_dictionary,interaction_dict,residue_list,interaction_type):
    fp_length = len(residue_dictionary)*len(interaction_dict) 
    fp = [0]*fp_length    
    for residue in residue_list:
        fp_index = residue_dictionary[residue][interaction_dict[interaction_type]]
        fp[fp_index] = fp[fp_index]+1
    return fp

# create fingerprints for all structures
fp_list = []
for structure, sites in interactions.items():
    print structure
    for site_name, site_interactions in sites.items():
        if not site_name.startswith('LIG'):
            continue  # fragments are labeled as LIG; other "sites" detected by PLIP are XRC artefacts
            
        print site_name
        for interaction_type, dataframe in site_to_dataframes(site_interactions).items():
            if dataframe is not None:
                residue_nos = dataframe['RESNR'].tolist()
                fp = create_fingerprint(residue_dict,interaction_dict,residue_nos,interaction_type)
                fp_list.append(fp)
        print fp
        

Mpro-x0946.pdb
LIG:A:1101
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### Overall Frequency fingerprints

In [35]:
len(fp_list)

48

In [36]:
# count fingerprint
count_fp = [sum(i) for i in zip(*fp_list)]
count_fp

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 3,
 0,
 1,
 9,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [37]:
# frequency fingerprint
frequency_fp = [float(i)/len(fp_list) for i in count_fp]
frequency_fp

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.041666666666666664,
 0.041666666666666664,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.041666666666666664,
 0.0625,
 0.0,
 0.020833333333333332,
 0.1875,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.020833333333333332,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.

## 2D Visualization
### Tabular 

### Wave 